In [ ]:
from presentation_utils import *
from DoD import view_4c_analysis_baseline as v4c
import glob
import pprint

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

# Config

In [ ]:
# directory storing the views
dir_path = "./toytest/"
# max size of candidate key
candidate_key_size = 2
# sample rows to present
sample_size = 5
# exploration / exploitation (choose next pair to present among top_k)
top_k = 10
# epsilon-greedy
epsilon = 0.1

max_num_interactions = 1000

# Run 4C

In [ ]:
# Run 4C
print("Running 4C...")

compatible_groups, contained_groups, complementary_groups, contradictory_groups, all_pair_contr_compl = \
    v4c.main(dir_path, candidate_key_size)

print("Compatible groups:")
for group in compatible_groups:
    print(group)

print("Contained groups:")
for group in contained_groups:
    print(group)

print("Contradictory groups:")
for path1, candidate_key_tuple, key_value_tuples, path2 in contradictory_groups:
    print(str(list(candidate_key_tuple)) + ": " + path1 + " - " + path2)

print("Complementary groups:")
for path1, path2, candidate_key_tuple, _, _ in complementary_groups:
    print(str(list(candidate_key_tuple)) + ": " + path1 + " - " + path2)

# Pruning 4C views
## Remove identical views and keep the contained view with the largest cardinality

In [ ]:
view_files = glob.glob(dir_path + "/view_*")
print("Number of views: ", len(view_files))

view_files = prune_compatible_views(view_files, compatible_groups)
print("After pruning compatible views: ", len(view_files))

view_files = prune_contained_views(view_files, contained_groups)
print("After pruning contained views: ", len(view_files))

# Pre-processing and generating sample rows to present

In [ ]:
print("Pre-processing...")

contr_or_compl_view_pairs, non_contr_or_compl_views, row_to_path_dict = preprocess(view_files, all_pair_contr_compl, sample_size)

# Actual presentation

In [ ]:
%gui asyncio
# Using asynchronous widgets. Requires ipykernel 4.7 or later
# pip install ipython ipykernel --upgrade

# async
task = present_async(view_files, contr_or_compl_view_pairs, non_contr_or_compl_views, row_to_path_dict,
                     top_k, epsilon, max_num_interactions)

# sync
# final_view_scores, num_interactions = present(view_files, contr_or_compl_view_pairs, non_contr_or_compl_views, row_to_path_dict,
#                                               top_k, epsilon, max_num_interactions)

# Final view scores

In [ ]:
# async
await task
final_view_scores, num_interactions = task.result()

print("Final view scores:")
pprint.pprint(final_view_scores)

print("Number of interactions = " + str(num_interactions))